# GSVA——基因集变异分析
> 本节概览：
> 1. GSVA简单介绍
> 2. 基因集的下载读取: 手动与msigdbr包下载
> 3. GSVA的运行
> 4. limma差异分析
> 5. GSVA结果可视化：热图、火山图、发散条形图/柱形偏差图


## 1. GSVA简单介绍
官方文档：[GSVA: gene set variation analysis (bioconductor.org)](https://link.zhihu.com/?target=https%3A//www.bioconductor.org/packages/release/bioc/vignettes/GSVA/inst/doc/GSVA.html)

不错的一篇文章：[GSVA的使用 - raisok](https://link.zhihu.com/?target=https%3A//www.cnblogs.com/raisok/p/11039239.html)

- 定义

**基因集变异分析(GSVA)**是一种特殊类型的基因集富集方法，通过对分析的功能单元进行概念上简单但功能强大的改变——从基因到基因集，从而实现对分子数据的路径中心分析。 简单来说，就是将分析对象由基因换成了基因集，进行基因集（通路）级别的差异分析。

- 原理和作用

通过将基因在不同样品间的表达量矩阵转化成基因集在样品间的表达量矩阵，从而来评估不同的通路在不同样品间是否富集。其实就是研究这些感兴趣的基因集在不同样品间的差异，或者寻找比较重要的基因集，作为一种分析方法，主要是是为了从生物信息学的角度去解释导致表型差异的原因。

### 分析前准备：

In [2]:
rm(list = ls())
options(stringsAsFactors = F)
library(tidyverse)
library(clusterProfiler)
library(msigdbr) # install.packages("msigdbr")
library(GSVA)
library(GSEABase)
library(pheatmap)
library(limma)
library(BiocParallel)

setwd("/slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/salmon")
load(file = "1.counts.Rdata") # 包含 tpm counts group_list gl
dir.create("6.GSVA")
setwd("6.GSVA")


Warning message in dir.create("6.GSVA"):
“'6.GSVA' already exists”


## 2.下载GSVA分析所需的基因集
GSVA分析常用MSigDB数据库中基因集，也可以自定义基因集进行分析。 MSigDB数据库目前有H和C1-C8九个定义的基因集，下面示范下载包含KEGG信息的C2与包含GO信息的C5基因集的两种方式——手动下载读取或msigdbr包下载提取。

<image src="../image/67.png">

### 2.1 手动下载读取
下载地址：[Downloads (gsea-msigdb.org)](https://link.zhihu.com/?target=http%3A//www.gsea-msigdb.org/gsea/downloads.jsp)

进入gsea-msigdb官网后可能还需要登录邮箱，然后找到需要下载的基因集下载，下载后将gmt文件放入指定文件夹，将其路径读取进入R即可。

不过需要注意的是这里的基因集默认都是人类的，如果是分析小鼠或其他物种最好采用MigDB包下载

<image src="../image/68.png">

In [ ]:
#### 对 MigDB( Molecular Signatures Database)中的基因集做GSVA分析  ####
## 用手动下载基因集做GSVA分析
d <- "path/gmt" # 存放gmt文件的路径
gmtfs <- list.files(d, pattern = "symbols.gmt") # 路径下所有结尾为symbols.gmt文件
gmtfs
kegg_list <- getGmt(file.path(d, gmtfs[1]))
go_list <- getGmt(file.path(d, gmtfs[2]))


### 2.2 msigdbr包下载读取
使用`msigdbr`包可以直接在R里下载`C2`和`C5`基因集，并提取相关信息做成list。 msigdbr下载数据可以指定物种，用msigdbr_species() 和 msigdbr_collections()可以查看支持的物种和基因集类别。

以下参考：[GSEA和GSVA，再也不用去下载gmt文件咯](https://link.zhihu.com/?target=https%3A//www.jianshu.com/p/0098baf2df46)

In [9]:
## msigdbr包提取下载 一般尝试KEGG和GO做GSVA分析
## KEGG
KEGG_df_all <- msigdbr(
    species = "Mus musculus", # Homo sapiens or Mus musculus
    category = "C2",
    subcategory = "CP:KEGG"
)
KEGG_df <- dplyr::select(
    KEGG_df_all,
    gs_name,
    gs_exact_source,
    gene_symbol
)
kegg_list <- split(
    KEGG_df$gene_symbol,
    KEGG_df$gs_name
) ## 按照gs_name给gene_symbol分组

## GO
GO_df_all <- msigdbr(
    species = "Mus musculus",
    category = "C5"
)
GO_df <- dplyr::select(
    GO_df_all,
    gs_name,
    gene_symbol,
    gs_exact_source,
    gs_subcat
)
GO_df <- GO_df[GO_df$gs_subcat != "HPO", ]
go_list <- split(
    GO_df$gene_symbol,
    GO_df$gs_name
) ## 按照gs_name给gene_symbol分组


In [10]:
head(kegg_list)
head(go_list)


$KEGG_ABC_TRANSPORTERS
 [1] "Abca1"   "Abca12"  "Abca13"  "Abca2"   "Abca3"   "Abca4"   "Abca5"  
 [8] "Abca6"   "Abca7"   "Abca8b"  "Abca9"   "Abcb1a"  "Abcb10"  "Abcb11" 
[15] "Abcb4"   "Abcb5"   "Abcb6"   "Abcb7"   "Abcb8"   "Abcb9"   "Abcc1"  
[22] "Abcc10"  "Abcc12"  "Gm10768" "Abcc3"   "Abcc4"   "Abcc5"   "Abcc6"  
[29] "Abcc8"   "Abcc9"   "Abcd1"   "Abcd2"   "Abcd3"   "Abcd4"   "Abcg1"  
[36] "Abcg2"   "Abcg4"   "Abcg5"   "Abcg8"   "Cftr"    "Tap1"    "Tap2"   

$KEGG_ACUTE_MYELOID_LEUKEMIA
 [1] "Akt1"     "Akt2"     "Akt3"     "Araf"     "Bad"      "Braf"    
 [7] "Ccna1"    "Ccnd1"    "Cebpa"    "Chuk"     "Eif4ebp1" "Flt3"    
[13] "Grb2"     "Hras"     "Ikbkb"    "Ikbkg"    "Jup"      "Kit"     
[19] "Kras"     "Lef1"     "Map2k1"   "Map2k2"   "Mapk1"    "Mapk3"   
[25] "Mtor"     "Myc"      "Nfkb1"    "Nras"     "Pik3ca"   "Pik3cb"  
[31] "Pik3cd"   "Pik3cg"   "Pik3r1"   "Pik3r2"   "Pik3r3"   "Pik3r5"  
[37] "Pim1"     "Pim2"     "Pml"      "Ppard"    "Raf1"     "Rara"    
[43] "Rela"     "Rps6kb1"  "Rps6kb2"  "Runx1"    "Runx1t1"  "Sos1"    
[49] "Sos2"     "Spi1"     "Stat3"    "Stat5a"   "Stat5b"   "Tcf7"    
[55] "Tcf7l1"   "Tcf7l2"   "Zbtb16"  

$KEGG_ADHERENS_JUNCTION
 [1] "Acp1"    "Actb"    "Actg1"   "Actn1"   "Actn2"   "Actn3"   "Actn4"  
 [8] "Afdn"    "Baiap2"  "Cdc42"   "Cdh1"    "Crebbp"  "Csnk2a1" "Csnk2a2"
[15] "Csnk2b"  "Ctnna1"  "Ctnna2"  "Ctnna3"  "Ctnnb1"  "Ctnnd1"  "Egfr"   
[22] "Ep300"   "Erbb2"   "Farp2"   "Fer"     "Fgfr1"   "Fyn"     "Igf1r"  
[29] "Insr"    "Iqgap1"  "Lef1"    "Lmo7"    "Map3k7"  "Mapk1"   "Mapk3"  
[36] "Met"     "Nectin1" "Nectin2" "Nectin3" "Nectin4" "Nlk"     "Pard3"  
[43] "Ptpn1"   "Ptpn6"   "Ptprb"   "Ptprf"   "Ptprj"   "Ptprm"   "Rac1"   
[50] "Rac2"    "Rac3"    "Rhoa"    "Smad2"   "Smad3"   "Smad4"   "Snai1"  
[57] "Snai2"   "Sorbs1"  "Src"     "Ssx2ip"  "Tcf7"    "Tcf7l1"  "Tcf7l2" 
[64] "Tgfbr1"  "Tgfbr2"  "Tjp1"    "Vcl"     "Was"     "Wasf1"   "Wasf2"  
[71] "Wasf3"   "Wasl"    "Yes1"   

$KEGG_ADIPOCYTOKINE_SIGNALING_PATHWAY
 [1] "Acacb"    "Acsl1"    "Acsl3"    "Acsl4"    "Acsl5"    "Acsl6"   
 [7] "Adipoq"   "Adipor1"  "Adipor2"  "Agrp"     "Akt1"     "Akt2"    
[13] "Akt3"     "Camkk1"   "Camkk2"   "Cd36"     "Chuk"     "Cpt1a"   
[19] "Cpt1b"    "Cpt1c"    "G6pc"     "G6pc2"    "Ikbkb"    "Ikbkg"   
[25] "Irs1"     "Irs2"     "Irs4"     "Jak2"     "Lep"      "Lepr"    
[31] "Mapk10"   "Mapk8"    "Mapk9"    "Mtor"     "Nfkb1"    "Nfkbia"  
[37] "Nfkbib"   "Nfkbie"   "Npy"      "Pck1"     "Pck2"     "Pomc"    
[43] "Ppara"    "Ppargc1a" "Prkaa1"   "Prkaa2"   "Prkab1"   "Prkab2"  
[49] "Prkag1"   "Prkag2"   "Prkag3"   "Prkcq"    "Ptpn11"   "Rela"    
[55] "Rxra"     "Rxrb"     "Rxrg"     "Slc2a1"   "Slc2a4"   "Socs3"   
[61] "Stat3"    "Stk11"    "Tnf"      "Tnfrsf1a" "Tnfrsf1b" "Tradd"   
[67] "Traf2"   

$KEGG_ALANINE_ASPARTATE_AND_GLUTAMATE_METABOLISM
 [1] "Abat"    "Acy3"    "Adsl"    "Adssl1"  "Adss"    "Agxt"    "Agxt2"  
 [8] "Aldh4a1" "Aldh5a1" "Asl"     "Asns"    "Aspa"    "Ass1"    "Cad"    
[15] "Cps1"    "Ddo"     "Gad1"    "Gad2"    "Gfpt1"   "Gfpt2"   "Gls"    
[22] "Gls2"    "Glud1"   "Glud1"   "Glul"    "Got1"    "Got2"    "Gpt"    
[29] "Gpt2"    "Il4i1b"  "Nit2"    "Ppat"   

$KEGG_ALDOSTERONE_REGULATED_SODIUM_REABSORPTION
 [1] "Atp1a1"   "Atp1a2"   "Atp1a3"   "Atp1a4"   "Atp1b1"   "Atp1b2"  
 [7] "Atp1b3"   "Atp1b4"   "Fxyd2"    "Fxyd4"    "Hsd11b1"  "Hsd11b2" 
[13] "Igf1"     "Ins2"     "Insr"     "Irs1"     "Irs2"     "Irs4"    
[19] "Kcnj1"    "Kras"     "Mapk1"    "Mapk3"    "Nedd4l"   "Nr3c2"   
[25] "Pdpk1"    "Pik3ca"   "Pik3cb"   "Pik3cd"   "Pik3cg"   "Pik3r1"  
[31] "Pik3r2"   "Pik3r3"   "Pik3r5"   "Prkca"    "Prkcb"    "Prkcg"   
[37] "Scnn1a"   "Scnn1b"   "Scnn1g"   "Sfn"      "Sgk1"     "Slc9a3r2"

$GOBP_10_FORMYLTETRAHYDROFOLATE_METABOLIC_PROCESS
[1] "Aasdhppt" "Aldh1l1"  "Aldh1l2"  "Mthfd1"   "Mthfd1l"  "Mthfd2l" 

$GOBP_2_OXOGLUTARATE_METABOLIC_PROCESS
 [1] "Aadat"  "Adhfe1" "D2hgdh" "Dlst"   "Got1"   "Got2"   "Gpt2"   "Idh1"  
 [9] "Idh2"   "Idh3b"  "Kyat3"  "L2hgdh" "Mrps36" "Ogdh"   "Ogdhl"  "Phyh"  
[17] "Tat"   

$GOBP_2FE_2S_CLUSTER_ASSEMBLY
[1] "Bola2" "Bola2" "Glrx3" "Glrx5" "Hscb"  "Nfs1" 

$GOBP_3_PHOSPHOADENOSINE_5_PHOSPHOSULFATE_BIOSYNTHETIC_PROCESS
[1] "Papss1"  "Papss2"  "Slc26a1" "Slc26a2" "Slc35b2" "Slc35b3"

$GOBP_3_PHOSPHOADENOSINE_5_PHOSPHOSULFATE_METABOLIC_PROCESS
 [1] "Abhd14b" "Bpnt1"   "Enpp1"   "Papss1"  "Papss2"  "Slc26a1" "Slc26a2"
 [8] "Slc35b2" "Slc35b3" "Sult1a1" "Sult1a1" "Sult1a1" "Sult1a1" "Sult1b1"
[15] "Sult1c1" "Sult1e1" "Sult2a1" "Sult2a5" "Sult2a6" "Sult2b1" "Tpst1"  
[22] "Tpst2"  

$GOBP_3_UTR_MEDIATED_MRNA_DESTABILIZATION
 [1] "Cpeb3"   "Dhx36"   "Dnd1"    "Hnrnpd"  "Khsrp"   "Mov10"   "Plekhn1"
 [8] "Rbm24"   "Rc3h1"   "Tardbp"  "Trim71"  "Upf1"    "Zc3h12a" "Zc3h12d"
[15] "Zfp36"   "Zfp36l1" "Zfp36l2"

## 3. GSVA的运行
使用GSVA需要输入基因表达矩阵和基因集。 基因集即为我们上一步所得`list`；基因表达矩阵可以使用 **`logCPM`、`logRPKM`、`logTPM`（GSVA参数kcdf选择"`Gaussian`"，默认）或`counts`数据（参数kcdf选择"`Poisson`"）**。 GSVA还支持BiocParallel，可设置参数parallel.sz进行多核计算。

下面选取基因集**go_list**和**logTPM**数据进行示范

In [11]:
ls()


[1] "counts"      "counts_raw"  "gl"          "GO_df"       "GO_df_all"  
 [6] "go_list"     "group_list"  "KEGG_df"     "KEGG_df_all" "kegg_list"  
[11] "tpm"         "txi"

In [18]:
args(gsva)


function (param, ...) 
NULL

In [38]:
####  GSVA  ####
# GSVA算法需要处理logCPM, logRPKM,logTPM数据或counts数据的矩阵####
# dat <- as.matrix(counts)
# dat <- as.matrix(log2(edgeR::cpm(counts))+1)
dat <- as.matrix(log2(tpm + 1))
geneset <- go_list
gsvapar <- gsvaParam(dat,
    geneset,
    kcdf = "Gaussian", # "Gaussian" for logCPM,logRPKM,logTPM, "Poisson" for counts
    maxDiff = TRUE,
)
gsva_mat <- GSVA::gsva(
    gsvapar,
)

write.csv(gsva_mat, "gsva_go_matrix.csv")


Warning message in .filterAndMapGenesAndGeneSets(param):
“Some gene sets have size one. Consider setting 'minSize' to a value > 1.”


Estimating GSVA scores for 10374 gene sets.
Estimating ECDFs with Gaussian kernels
  |                                                                      |   0%

  |======================================================================| 100%



运行完GSVA后gsva_mat内容如下，可以发现行名变成了基因集通路名，每个样品都会有对应通路的GSVA评分：

<image src="../image/69.png">

## 4. limma差异分析
得到GSVA评分的矩阵后，我们需要利用limma包进行pathway通路的差异分析，与之前介绍的基因差异分析流程类似，**但不需要进行 limma-trend 或 voom的步骤**

In [43]:
## 手动修正

#### 进行limma差异处理 ####
## 设定 实验组exp / 对照组ctr
gl
exp <- "primed"
ctr <- "naive"
group_list <- c("naive", "naive", "primed")
gl$group_list <- group_list
design <- model.matrix(~ 0 + factor(group_list))
colnames(gsva_mat) <- c("RNA_mESCs_1", "RNA_mESCs_2", "RNA_mESCs_3")
gl
group_list
design


,group_list
,<chr>
RNA_mESCs_1,naive
RNA_mESCs_2,naive
RNA_mESCs_3,primed


,group_list
,<chr>
RNA_mESCs_1,naive
RNA_mESCs_2,naive
RNA_mESCs_3,primed


[1] "naive"  "naive"  "primed"

,factor(group_list)naive,factor(group_list)primed
1,1,0
2,1,0
3,0,1


In [44]:
colnames(design) <- levels(factor(group_list))
rownames(design) <- colnames(gsva_mat)
design


,naive,primed
RNA_mESCs_1,1,0
RNA_mESCs_2,1,0
RNA_mESCs_3,0,1


In [45]:
contrast.matrix <- makeContrasts(
    contrasts = paste0(exp, "-", ctr), # "exp/ctrl"
    levels = design
)

fit1 <- lmFit(gsva_mat, design) # 拟合模型
fit2 <- contrasts.fit(fit1, contrast.matrix) # 统计检验
efit <- eBayes(fit2) # 修正

summary(decideTests(efit, lfc = 1, p.value = 0.05)) # 统计查看差异结果
tempOutput <- topTable(efit, coef = paste0(exp, "-", ctr), n = Inf)
degs <- na.omit(tempOutput)
write.csv(degs, "gsva_go_degs.results.csv")


       primed-naive
Down              0
NotSig        10374
Up                0

## 5. GSVA结果可视化：热图、火山图、发散条形图/柱形偏差图
与常规差异分析结果展示类似，GSVA结果可视化一般也用热图、火山图展示
### 5.1 热图

```R
#### 对GSVA的差异分析结果进行热图可视化 #### 
##设置筛选阈值
padj_cutoff=0.05
log2FC_cutoff=log2(2)

keep <- rownames(degs[degs$adj.P.Val < padj_cutoff & abs(degs$logFC)>log2FC_cutoff, ])
length(keep)
dat <- gsva_mat[keep[1:50],] #选取前50进行展示

pheatmap::pheatmap(dat, 
                   fontsize_row = 8,
                   height = 10,
                   width=18,
                   annotation_col = gl,
                   show_colnames = F,
                   show_rownames = T,
                   filename = paste0('GSVA_go_heatmap.pdf'))
```

In [49]:
head(gsva_mat)


,RNA_mESCs_1,RNA_mESCs_2,RNA_mESCs_3
GOBP_10_FORMYLTETRAHYDROFOLATE_METABOLIC_PROCESS,0.06172880,-0.152024140,-0.02425856
GOBP_2_OXOGLUTARATE_METABOLIC_PROCESS,-0.15820086,0.378670836,-0.37987839
GOBP_2FE_2S_CLUSTER_ASSEMBLY,0.14971505,0.280077228,-0.81990669
GOBP_3_PHOSPHOADENOSINE_5_PHOSPHOSULFATE_BIOSYNTHETIC_PROCESS,0.16556400,-0.216962078,0.10313275
GOBP_3_PHOSPHOADENOSINE_5_PHOSPHOSULFATE_METABOLIC_PROCESS,-0.05181735,-0.008918808,-0.16060131
GOBP_3_UTR_MEDIATED_MRNA_DESTABILIZATION,-0.07552885,-0.500450175,0.20998596


In [51]:
#### 对GSVA的差异分析结果进行热图可视化 ####
## 设置筛选阈值
padj_cutoff <- 0.1
log2FC_cutoff <- log2(2)

keep <- rownames(degs[degs$adj.P.Val < padj_cutoff & abs(degs$logFC) > log2FC_cutoff, ])
length(keep)
dat <- gsva_mat[keep[1:50], ] # 选取前50进行展示

pheatmap::pheatmap(dat,
    fontsize_row = 8,
    height = 10,
    width = 18,
    annotation_col = gl,
    show_colnames = F,
    show_rownames = T,
    filename = paste0("GSVA_go_heatmap.pdf")
)


[1] 119

### 5.2 火山图
```R
degs$significance  <- as.factor(ifelse(degs$adj.P.Val < padj_cutoff & abs(degs$logFC) > log2FC_cutoff,
                                           ifelse(degs$logFC > log2FC_cutoff ,'UP','DOWN'),'NOT'))

this_title <- paste0(' Up :  ',nrow(degs[degs$significance =='UP',]) ,
                     '\n Down : ',nrow(degs[degs$significance =='DOWN',]),
                     '\n adj.P.Val <= ',padj_cutoff,
                     '\n FoldChange >= ',round(2^log2FC_cutoff,3))

g <- ggplot(data=degs, 
            aes(x=logFC, y=-log10(adj.P.Val),
                color=significance)) +
  #点和背景
  geom_point(alpha=0.4, size=1) +
  theme_classic()+ #无网格线
  #坐标轴
  xlab("log2 ( FoldChange )") + 
  ylab("-log10 ( adj.P.Val )") +
  #标题文本
  ggtitle( this_title ) +
  #分区颜色                   
  scale_colour_manual(values = c('blue','grey','red'))+ 
  #辅助线
  geom_vline(xintercept = c(-log2FC_cutoff,log2FC_cutoff),lty=4,col="grey",lwd=0.8) +
  geom_hline(yintercept = -log10(padj_cutoff),lty=4,col="grey",lwd=0.8) +
  #图例标题间距等设置
  theme(plot.title = element_text(hjust = 0.5), 
        plot.margin=unit(c(2,2,2,2),'lines'), #上右下左
        legend.title = element_blank(),
        legend.position="right")

ggsave(g,filename = 'GSVA_go_volcano_padj.pdf',width =8,height =7.5)
```

In [52]:
degs$significance <- as.factor(ifelse(degs$adj.P.Val < padj_cutoff & abs(degs$logFC) > log2FC_cutoff,
    ifelse(degs$logFC > log2FC_cutoff, "UP", "DOWN"), "NOT"
))

this_title <- paste0(
    " Up :  ", nrow(degs[degs$significance == "UP", ]),
    "\n Down : ", nrow(degs[degs$significance == "DOWN", ]),
    "\n adj.P.Val <= ", padj_cutoff,
    "\n FoldChange >= ", round(2^log2FC_cutoff, 3)
)

g <- ggplot(
    data = degs,
    aes(
        x = logFC, y = -log10(adj.P.Val),
        color = significance
    )
) +
    # 点和背景
    geom_point(alpha = 0.4, size = 1) +
    theme_classic() + # 无网格线
    # 坐标轴
    xlab("log2 ( FoldChange )") +
    ylab("-log10 ( adj.P.Val )") +
    # 标题文本
    ggtitle(this_title) +
    # 分区颜色
    scale_colour_manual(values = c("blue", "grey", "red")) +
    # 辅助线
    geom_vline(xintercept = c(-log2FC_cutoff, log2FC_cutoff), lty = 4, col = "grey", lwd = 0.8) +
    geom_hline(yintercept = -log10(padj_cutoff), lty = 4, col = "grey", lwd = 0.8) +
    # 图例标题间距等设置
    theme(
        plot.title = element_text(hjust = 0.5),
        plot.margin = unit(c(2, 2, 2, 2), "lines"), # 上右下左
        legend.title = element_blank(),
        legend.position = "right"
    )

ggsave(g, filename = "GSVA_go_volcano_padj.pdf", width = 8, height = 7.5)


### 5.3 发散条形图/柱形偏差图
为了更好展示绘制发`散条形图/柱形偏差图`，此处用的是`KEGG`的`gsva`差异分析结果，展示通路的上下调及pvalue信息（也可以是t值或padj值等），详细[绘图过程见发散条形图/柱形偏差图 - 简书 (jianshu.com)](https://link.zhihu.com/?target=https%3A//www.jianshu.com/p/cf23a9c6bb9c)


```R
#### 发散条形图绘制 ####
library(tidyverse)  # ggplot2 stringer dplyr tidyr readr purrr  tibble forcats
library(ggthemes)
library(ggprism)
p_cutoff=0.001
degs <- gsva_kegg_degs  #载入gsva的差异分析结果
Diff <- rbind(subset(degs,logFC>0)[1:20,], subset(degs,logFC<0)[1:20,]) #选择上下调前20通路     
dat_plot <- data.frame(id  = row.names(Diff),
                       p   = Diff$P.Value,
                       lgfc= Diff$logFC)
dat_plot$group <- ifelse(dat_plot$lgfc>0 ,1,-1)    # 将上调设为组1，下调设为组-1
dat_plot$lg_p <- -log10(dat_plot$p)*dat_plot$group # 将上调-log10p设置为正，下调-log10p设置为负
dat_plot$id <- str_replace(dat_plot$id, "KEGG_","");dat_plot$id[1:10]
dat_plot$threshold <- factor(ifelse(abs(dat_plot$p) <= p_cutoff,
                                   ifelse(dat_plot$lgfc >0 ,'Up','Down'),'Not'),
                            levels=c('Up','Down','Not'))

dat_plot <- dat_plot %>% arrange(lg_p)
dat_plot$id <- factor(dat_plot$id,levels = dat_plot$id)

## 设置不同标签数量
low1 <- dat_plot %>% filter(lg_p < log10(p_cutoff)) %>% nrow()
low0 <- dat_plot %>% filter(lg_p < 0) %>% nrow()
high0 <- dat_plot %>% filter(lg_p < -log10(p_cutoff)) %>% nrow()
high1 <- nrow(dat_plot)

p <- ggplot(data = dat_plot,aes(x = id, y = lg_p, 
                                fill = threshold)) +
  geom_col()+
  coord_flip() + 
  scale_fill_manual(values = c('Up'= '#36638a','Not'='#cccccc','Down'='#7bcd7b')) +
  geom_hline(yintercept = c(-log10(p_cutoff),log10(p_cutoff)),color = 'white',size = 0.5,lty='dashed') +
  xlab('') + 
  ylab('-log10(P.Value) of GSVA score') + 
guides(fill="none")+
  theme_prism(border = T) +
  theme(
    axis.text.y = element_blank(),
    axis.ticks.y = element_blank()
  ) +
  geom_text(data = dat_plot[1:low1,],aes(x = id,y = 0.1,label = id),
              hjust = 0,color = 'black') + #黑色标签
  geom_text(data = dat_plot[(low1 +1):low0,],aes(x = id,y = 0.1,label = id),
            hjust = 0,color = 'grey') + # 灰色标签
  geom_text(data = dat_plot[(low0 + 1):high0,],aes(x = id,y = -0.1,label = id),
            hjust = 1,color = 'grey') + # 灰色标签
  geom_text(data = dat_plot[(high0 +1):high1,],aes(x = id,y = -0.1,label = id),
            hjust = 1,color = 'black') # 黑色标签

ggsave("GSVA_barplot_pvalue.pdf",p,width = 15,height  = 15)
```

In [56]:
#### 发散条形图绘制 ####
library(tidyverse) # ggplot2 stringer dplyr tidyr readr purrr  tibble forcats
library(ggthemes)
library(ggprism)
p_cutoff <- 0.001
# degs <- gsva_kegg_degs # 载入gsva的差异分析结果
degs <- degs # 载入gsva的差异分析结果
Diff <- rbind(subset(degs, logFC > 0)[1:20, ], subset(degs, logFC < 0)[1:20, ]) # 选择上下调前20通路
dat_plot <- data.frame(
  id = row.names(Diff),
  p = Diff$P.Value,
  lgfc = Diff$logFC
)
dat_plot$group <- ifelse(dat_plot$lgfc > 0, 1, -1) # 将上调设为组1，下调设为组-1
dat_plot$lg_p <- -log10(dat_plot$p) * dat_plot$group # 将上调-log10p设置为正，下调-log10p设置为负
dat_plot$id <- str_replace(dat_plot$id, "KEGG_", "")
dat_plot$id[1:10]
dat_plot$threshold <- factor(
  ifelse(abs(dat_plot$p) <= p_cutoff,
    ifelse(dat_plot$lgfc > 0, "Up", "Down"), "Not"
  ),
  levels = c("Up", "Down", "Not")
)

dat_plot <- dat_plot %>% arrange(lg_p)
dat_plot$id <- factor(dat_plot$id, levels = dat_plot$id)

## 设置不同标签数量
low1 <- dat_plot %>%
  filter(lg_p < log10(p_cutoff)) %>%
  nrow()
low0 <- dat_plot %>%
  filter(lg_p < 0) %>%
  nrow()
high0 <- dat_plot %>%
  filter(lg_p < -log10(p_cutoff)) %>%
  nrow()
high1 <- nrow(dat_plot)

p <- ggplot(data = dat_plot, aes(
  x = id, y = lg_p,
  fill = threshold
)) +
  geom_col() +
  coord_flip() +
  scale_fill_manual(values = c("Up" = "#36638a", "Not" = "#cccccc", "Down" = "#7bcd7b")) +
  geom_hline(yintercept = c(-log10(p_cutoff), log10(p_cutoff)), color = "white", size = 0.5, lty = "dashed") +
  xlab("") +
  ylab("-log10(P.Value) of GSVA score") +
  guides(fill = "none") +
  theme_prism(border = T) +
  theme(
    axis.text.y = element_blank(),
    axis.ticks.y = element_blank()
  ) +
  geom_text(
    data = dat_plot[1:low1, ], aes(x = id, y = 0.1, label = id),
    hjust = 0, color = "black"
  ) + # 黑色标签
  geom_text(
    data = dat_plot[(low1 + 1):low0, ], aes(x = id, y = 0.1, label = id),
    hjust = 0, color = "grey"
  ) + # 灰色标签
  geom_text(
    data = dat_plot[(low0 + 1):high0, ], aes(x = id, y = -0.1, label = id),
    hjust = 1, color = "grey"
  ) + # 灰色标签
  geom_text(
    data = dat_plot[(high0 + 1):high1, ], aes(x = id, y = -0.1, label = id),
    hjust = 1, color = "black"
  ) # 黑色标签

ggsave("GSVA_barplot_pvalue.pdf", p, width = 15, height = 15)


[1] "GOBP_REGULATION_OF_RESPONSE_TO_DRUG"                                                                            
 [2] "GOBP_RENAL_CONTROL_OF_PERIPHERAL_VASCULAR_RESISTANCE_INVOLVED_IN_REGULATION_OF_SYSTEMIC_ARTERIAL_BLOOD_PRESSURE"
 [3] "GOMF_PRE_MRNA_5_SPLICE_SITE_BINDING"                                                                            
 [4] "GOBP_REGULATION_OF_INTEGRIN_BIOSYNTHETIC_PROCESS"                                                               
 [5] "GOBP_POSITIVE_REGULATION_OF_TRANSCRIPTION_FROM_RNA_POLYMERASE_II_PROMOTER_INVOLVED_IN_HEART_DEVELOPMENT"        
 [6] "GOCC_PROTEIN_COMPLEX_INVOLVED_IN_CELL_MATRIX_ADHESION"                                                          
 [7] "GOBP_TRANS_SYNAPTIC_SIGNALING_BY_LIPID"                                                                         
 [8] "GOBP_RETROGRADE_TRANS_SYNAPTIC_SIGNALING_BY_LIPID"                                                              
 [9] "GOBP_CORTICOTROPIN_SECRETION"                                                                                   
[10] "GOBP_REGULATION_OF_CORTICOTROPIN_SECRETION"

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
